# Plot SDS data availability

In [ ]:
import header
paths = header.setup_environment()
import os
#import sys
#import glob
#import numpy as np
import pandas as pd
#from obspy.core import read, Stream, UTCDateTime
#import FDSNtools
#import wrappers
import SDS
import libWellData as LLE

transducersDF = LLE.get_transducers_dataframe(paths)
#print(transducersDF)

# Availability by day
startdate = UTCDateTime(2022,7,21)
enddate = UTCDateTime(2022,12,3)
sdsobj = SDS.SDSobj(paths['SDS_TOP'])
trace_ids = None

availabilityCSV = os.path.join(paths['outdir'], 'availability4.csv')
if os.path.isfile(availabilityCSV):
    availabilityDF = pd.read_csv(availabilityCSV, index_col=None)
    trace_ids = availabilityDF.columns[1:]
else:
    availabilityDF, trace_ids = sdsobj._sds_percent_available_per_day(startdate, enddate, trace_ids=trace_ids, speed=3)
    availabilityDF.to_csv(availabilityCSV, index=False)

#cols = availabilityDF.columns
#print(cols)
#print(trace_ids)

def reorder_trace_ids(df, ordered_ids):
    print(ordered_ids)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    d = df['date']
    df = df.loc[:, ordered_ids]
    df.insert(0, 'date', d)
    #df['date'] = d
    return df

well_availabilityDF = availabilityDF.copy()
for id in trace_ids:
    if id[0:2] =='FL':
        well_availabilityDF.drop(labels=id, axis=1, inplace=True)
well_availabilityDF = reorder_trace_ids(well_availabilityDF, transducersDF['id'].to_list())
#print(well_availabilityDF.columns)
sdsobj.plot_availability(well_availabilityDF, outfile=os.path.join(paths['outdir'],'well_availability4.png'), labels=transducersDF.serial.to_list())

SA_availabilityDF = availabilityDF.copy()
for id in trace_ids:
    if id[0:2] !='FL':
        SA_availabilityDF.drop(labels=id, axis=1, inplace=True)
#print(SA_availabilityDF)
sdsobj.plot_availability(SA_availabilityDF, outfile=os.path.join(paths['outdir'],'SA_availability4.png'))

Falcon 9 Block 5 | Starlink Group 4-35		September 24, 2022	SLC-40	23:32:10 UTC